In [2]:
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

In [3]:
# Видео поток
# ipython notebook должен лежать в папке с картинками

videoCapture = cv2.VideoCapture("%02d.tif")

In [4]:
# Считывание кадра

def readFrame(videoCapture):
    retVal, frame = videoCapture.read()
    if retVal == False:
        return False, None
    else:
        # Так как видео уже в grayscale, то нужно просто дропнуть лишние каналы
        frameGray, _, _ = cv2.split(frame)
        return True, frameGray

In [7]:
# SHI TOMASI

shiTomasiDetector = cv2.GFTTDetector_create()
descriptor = cv2.xfeatures2d.FREAK_create()
def shitomasi(frame):
    kp = shiTomasiDetector.detect(frame)
    return descriptor.compute(frame, kp)

AttributeError: 'module' object has no attribute 'GFTTDetector_create'

In [5]:
# BRISK

briskDetector = cv2.BRISK_create()
def brisk(frame):
    kp = briskDetector.detect(frame)
    return briskDetector.compute(frame, kp)

In [6]:
# SIFT

siftDetector = cv2.xfeatures2d.SIFT_create()
def sift(frame):
    kp = siftDetector.detect(frame)
    return siftDetector.compute(frame, kp)

In [7]:
# Save Keypoints to image

def save(keypoints, label, frame, frameIdx):
    img = cv2.drawKeypoints(frame, keypoints, 0, color=(0,255,0), flags=0)
    cv2.imwrite(label + "/" + str(frameIdx) + ".png", img)

In [8]:
shitomasiLabel = "shitomasi"
briskLabel = "brisk"
siftLabel = "sift"

In [9]:
#bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = False)

#index_params = dict(algorithm = 0, trees = 5)
index_params= dict(algorithm = 6, table_number = 6, key_size = 12, multi_probe_level = 1)
search_params = dict(checks = 50)
matcher = cv2.FlannBasedMatcher(index_params, search_params)

def ratioOfMatchingDescriptors(desc1, desc2, idx):
    matches = matcher.match(desc1, desc2)
    return len(matches) / (len(desc1) + len(desc2)), str(idx)

def createPlotFromAllDescriptors(allDesc1, allDesc2, label):
    N = len(allDesc1)
    
    ratios = [ratioOfMatchingDescriptors(d1, d2, 0) for d1, d2 in zip(allDesc1, allDesc2)]
    print(ratios)
    #plt.plot(range(1, N + 1), elapsedTime)
   # plt.ylabel('время, c')
    #plt.xlabel('кадр')
   # plt.axis([1, 12, 0, 0.015])
   # plt.show()

In [10]:
def processFrames(videoCapture):
    shitomasiAllDesc = []
    briskAllDesc = []
    siftAllDesc = []
    
    idx = 0
    while(True):
        idx += 1
        retVal, frame = readFrame(videoCapture)
        if retVal == False:
            break
        
        print(idx)
        
        shitomasiKP, shitomasiDesc = shitomasi(frame)
        save(shitomasiKP, shitomasiLabel, frame, idx)
        
        briskKP, briskDesc = brisk(frame)
        save(briskKP, briskLabel, frame, idx)
        
        siftKP, siftDesc = sift(frame)
        save(siftKP, siftLabel, frame, idx)
        
        shitomasiAllDesc.append(shitomasiDesc)
        briskAllDesc.append(briskDesc)
        siftAllDesc.append(siftDesc)
        
    createPlotFromAllDescriptors(shitomasiAllDesc, shitomasiAllDesc, shitomasiLabel + briskLabel)
    createPlotFromAllDescriptors(shitomasiAllDesc, siftAllDesc, shitomasiLabel + siftLabel)
    createPlotFromAllDescriptors(siftAllDesc, briskAllDesc, siftLabel + briskLabel)

In [11]:
processFrames(videoCapture)

1
2
3
4
5
6
7
8
9
10
11
12


error: /Users/jenkins/miniconda/1/x64/conda-bld/conda_1486587097465/work/opencv-3.1.0/modules/python/src2/cv2.cpp:163: error: (-215) The data should normally be NULL! in function allocate
